<a href="https://colab.research.google.com/github/DinhIchMinhHoang/btl/blob/main/viXTTS_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔥🔥🔥**viXTTS Demo**🗣️🗣️🗣️

Demo này giúp bạn chạy viXTTS miễn phí trên Google Colab!
Xem thông tin mô hình tại [đây](https://huggingface.co/capleaf/viXTTS)

Bạn có thể dùng demo này với mục đích:
- ✅Mục đích cá nhân, học tập, nghiên cứu, thử nghiệm

Bạn **KHÔNG** được dùng demo này với mục đích:
- ❌Mục đích trái đạo đức, vi phạm pháp luật Việt Nam
- ❌Tạo ra nội dung gây thù ghét, kỳ thị, bạo lực hoặc nội dung vi phạm bản quyền
- ❌Giả mạo danh tính hoặc gây hiểu nhầm rằng nội dung được tạo ra bởi một cá nhân hoặc tổ chức khác

In [1]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://cli.github.com/packages stable InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,961 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,623 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy InReleas

In [ ]:
# Cập nhật pip & công cụ build
!pip install --upgrade pip setuptools wheel

# Cài maturin trước để tránh lỗi build Rust module
!pip install maturin

# Clone repo viXTTS từ nhánh add-vietnamese-xtts
!rm -rf TTS
!git clone --branch add-vietnamese-xtts https://github.com/thinhlpg/TTS.git
%cd TTS

# Cài đặt các dependency
!pip install -r requirements.txt

# Cài TTS ở chế độ editable
!pip install -e .

# Fix conflict websockets với Colab
!pip install websockets==13.0.1

# Cài đặt các thư viện bổ sung cần cho viXTTS
!pip install deepspeed vinorm==2.0.7 cutlet unidic==1.1.0 underthesea gradio==4.35 deepfilternet==0.5.6

# Tải bộ từ điển tiếng Nhật cho cutlet/unidic (bắt buộc)
import os
os.system("python -m unidic download")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 133.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [maturin]
Cloning into 'TTS'...
remote: Enumerating objects: 28904, done.
remote: Total 28904 (delta 0), reused 0 (delta 0), pack-reused 28904 (from 1)
Receiving objects: 100% (28904/28904), 136.14 MiB | 28.67 MiB/s, done.
Resolving deltas: 100% (20953/20953), done.
/content/TTS
Ignoring numpy: markers 'python_version > "3.10"' don't match your environment
Ignoring numba: markers 'python_version < "3.9"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 98.2 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 129.7 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 75.4 MB/s  0:00:00
  

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="thinhlpg/viXTTS",
    repo_type="model",
    local_dir="model"
)


In [ ]:
import os
from google.colab import files
from IPython.display import Audio, clear_output

# Bật tắt lọc nhiễu (DeepFilterNet)
denoise = False  # True = lọc nhiễu, False = giữ nguyên

uploaded = files.upload()
for filename in uploaded.keys():
    if denoise:
        !deepFilter --model DeepFilterNet3 "{filename}"
        !ffmpeg -i "{filename.replace('.wav', '_DeepFilterNet3.wav')}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
        os.remove(filename.replace('.wav', '_DeepFilterNet3.wav'))
    else:
        !ffmpeg -i "{filename}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
    os.remove(filename)
    break

clear_output()
print("✅ Đã tải file âm thanh thành công")
Audio("/content/model/user_sample.wav")


In [1]:
# The inference code is adopted from https://github.com/coqui-ai/TTS/blob/dev/TTS/demos/xtts_ft_demo/xtts_demo.py
# @title 2. 🤗 **Sử dụng**
# @markdown 👈 Nhấn để chạy.
# @markdown Nếu gặp lỗi thì cũng nhấn nút này nhé!

# @markdown Lần đầu chạy sẽ hơi lâu, bạn chờ tí nhé!

# @markdown Kết quả sẽ được lưu vào `/content/output`

# @markdown Chọn ngôn ngữ:
language = "Tiếng Việt" # @param ["Tiếng Việt", "Tiếng Anh","Tiếng Tây Ban Nha", "Tiếng Pháp","Tiếng Đức","Tiếng Ý", "Tiếng Bồ Đào Nha", "Tiếng Ba Lan", "Tiếng Thổ Nhĩ Kỳ", "Tiếng Nga", "Tiếng Hà Lan", "Tiếng Séc", "Tiếng Ả Rập", "Tiếng Trung (giản thể)", "Tiếng Nhật", "Tiếng Hungary", "Tiếng Hàn", "Tiếng Hindi"]
# @markdown Văn bản để đọc. Độ dài tối thiểu mỗi câu nên từ 10 từ để đặt kết quả tốt nhất.
input_text ="Xin chào, tôi là một công cụ có khả năng chuyển đổi văn bản thành giọng nói tự nhiên, được phát triển bởi nhóm Nón lá. Tôi có thể hổ trợ người khiếm thị,  đọc sách nói, làm trợ lý ảo, review phim, làm waifu để an ủi bạn, và phục vụ nhiều mục đích khác." # @param {type:"string"}
# @markdown Chọn giọng mẫu:
reference_audio = "model/vi_sample.wav" # @param [ "model/user_sample.wav",  "model/vi_sample.wav",  "model/samples/nam-calm.wav",  "model/samples/nam-cham.wav",  "model/samples/nam-nhanh.wav",  "model/samples/nam-truyen-cam.wav",  "model/samples/nu-calm.wav",  "model/samples/nu-cham.wav",  "model/samples/nu-luu-loat.wav",  "model/samples/nu-nhan-nha.wav",  "model/samples/nu-nhe-nhang.wav"]
# @markdown Tự động chuẩn hóa chữ (VD: 20/11 -> hai mươi tháng mười một)
normalize_text = True # @param {type:"boolean"}
# @markdown In chi tiết xử lý
verbose = True # @param {type:"boolean"}
# @markdown Lưu từng câu thành file riêng lẻ.
output_chunks = True # @param {type:"boolean"}

from IPython.display import clear_output
def cry_and_quit():
    clear_output()
    print("> Lỗi rồi huhu 😭😭, bạn hãy nhấn chạy lại phần này nhé!")
    quit()

import os
import string
import unicodedata
from datetime import datetime
from pprint import pprint

import torch
import torchaudio
from tqdm import tqdm
from underthesea import sent_tokenize
from unidecode import unidecode

try:
    from vinorm import TTSnorm
    from TTS.tts.configs.xtts_config import XttsConfig
    from TTS.tts.models.xtts import Xtts
except:
    cry_and_quit()

# Load model
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def load_model(xtts_checkpoint, xtts_config, xtts_vocab):
    clear_gpu_cache()
    if not xtts_checkpoint or not xtts_config or not xtts_vocab:
        return "You need to run the previous steps or manually set the `XTTS checkpoint path`, `XTTS config path`, and `XTTS vocab path` fields !!"
    config = XttsConfig()
    config.load_json(xtts_config)
    XTTS_MODEL = Xtts.init_from_config(config)
    print("Loading XTTS model! ")
    XTTS_MODEL.load_checkpoint(config,
                               checkpoint_path=xtts_checkpoint,
                               vocab_path=xtts_vocab,
                               use_deepspeed=True)
    if torch.cuda.is_available():
        XTTS_MODEL.cuda()

    print("Model Loaded!")
    return XTTS_MODEL


def get_file_name(text, max_char=50):
    filename = text[:max_char]
    filename = filename.lower()
    filename = filename.replace(" ", "_")
    filename = filename.translate(str.maketrans("", "", string.punctuation.replace("_", "")))
    filename = unidecode(filename)
    current_datetime = datetime.now().strftime("%m%d%H%M%S")
    filename = f"{current_datetime}_{filename}"
    return filename


def calculate_keep_len(text, lang):
    if lang in ["ja", "zh-cn"]:
        return -1

    word_count = len(text.split())
    num_punct = (
        text.count(".")
        + text.count("!")
        + text.count("?")
        + text.count(",")
    )

    if word_count < 5:
        return 15000 * word_count + 2000 * num_punct
    elif word_count < 10:
        return 13000 * word_count + 2000 * num_punct
    return -1


def normalize_vietnamese_text(text):
    text = (
        TTSnorm(text, unknown=False, lower=False, rule=True)
        .replace("..", ".")
        .replace("!.", "!")
        .replace("?.", "?")
        .replace(" .", ".")
        .replace(" ,", ",")
        .replace('"', "")
        .replace("'", "")
        .replace("AI", "Ây Ai")
        .replace("A.I", "Ây Ai")
    )
    return text


def run_tts(XTTS_MODEL, lang, tts_text, speaker_audio_file,
            normalize_text= True,
            verbose=False,
            output_chunks=False):
    """
    Run text-to-speech (TTS) synthesis using the provided XTTS_MODEL.

    Args:
        XTTS_MODEL: A pre-trained TTS model.
        lang (str): The language of the input text.
        tts_text (str): The text to be synthesized into speech.
        speaker_audio_file (str): Path to the audio file of the speaker to condition the synthesis on.
        normalize_text (bool, optional): Whether to normalize the input text. Defaults to True.
        verbose (bool, optional): Whether to print verbose information. Defaults to False.
        output_chunks (bool, optional): Whether to save synthesized speech chunks separately. Defaults to False.

    Returns:
        str: Path to the synthesized audio file.
    """

    if XTTS_MODEL is None or not speaker_audio_file:
        return "You need to run the previous step to load the model !!", None, None

    output_dir = "./output"
    os.makedirs(output_dir, exist_ok=True)

    gpt_cond_latent, speaker_embedding = XTTS_MODEL.get_conditioning_latents(
        audio_path=speaker_audio_file,
        gpt_cond_len=XTTS_MODEL.config.gpt_cond_len,
        max_ref_length=XTTS_MODEL.config.max_ref_len,
        sound_norm_refs=XTTS_MODEL.config.sound_norm_refs,
    )

    if normalize_text and lang == "vi":
        # Bug on google colab
        try:
            tts_text = normalize_vietnamese_text(tts_text)
        except:
            cry_and_quit()

    if lang in ["ja", "zh-cn"]:
        tts_texts = tts_text.split("。")
    else:
        tts_texts = sent_tokenize(tts_text)

    if verbose:
        print("Text for TTS:")
        pprint(tts_texts)

    wav_chunks = []
    for text in tqdm(tts_texts):
        if text.strip() == "":
            continue

        wav_chunk = XTTS_MODEL.inference(
            text=text,
            language=lang,
            gpt_cond_latent=gpt_cond_latent,
            speaker_embedding=speaker_embedding,
            temperature=0.3,
            length_penalty=1.0,
            repetition_penalty=10.0,
            top_k=30,
            top_p=0.85,
        )

        # Quick hack for short sentences
        keep_len = calculate_keep_len(text, lang)
        wav_chunk["wav"] = torch.tensor(wav_chunk["wav"][:keep_len])

        if output_chunks:
            out_path = os.path.join(output_dir, f"{get_file_name(text)}.wav")
            torchaudio.save(out_path, wav_chunk["wav"].unsqueeze(0), 24000)
            if verbose:
                print(f"Saved chunk to {out_path}")

        wav_chunks.append(wav_chunk["wav"])

    out_wav = torch.cat(wav_chunks, dim=0).unsqueeze(0)
    out_path = os.path.join(output_dir, f"{get_file_name(tts_text)}.wav")
    torchaudio.save(out_path, out_wav, 24000)

    if verbose:
        print(f"Saved final file to {out_path}")

    return out_path


language_code_map = {
    "Tiếng Việt": "vi",
    "Tiếng Anh": "en",
    "Tiếng Tây Ban Nha": "es",
    "Tiếng Pháp": "fr",
    "Tiếng Đức": "de",
    "Tiếng Ý": "it",
    "Tiếng Bồ Đào Nha": "pt",
    "Tiếng Ba Lan": "pl",
    "Tiếng Thổ Nhĩ Kỳ": "tr",
    "Tiếng Nga": "ru",
    "Tiếng Hà Lan": "nl",
    "Tiếng Séc": "cs",
    "Tiếng Ả Rập": "ar",
    "Tiếng Trung (giản thể)": "zh-cn",
    "Tiếng Nhật": "ja",
    "Tiếng Hungary": "hu",
    "Tiếng Hàn": "ko",
    "Tiếng Hindi": "hi"
}

print("> Đang nạp mô hình...")
try:
    if not vixtts_model:
        vixtts_model = load_model(xtts_checkpoint="model/model.pth",
                                xtts_config="model/config.json",
                                xtts_vocab="model/vocab.json")
except:
    vixtts_model = load_model(xtts_checkpoint="model/model.pth",
                                xtts_config="model/config.json",
                                xtts_vocab="model/vocab.json")
clear_output()
print("> Đã nạp mô hình")

if not os.path.exists(reference_audio):
    print("⚠️⚠️⚠️Bạn chưa tải file âm thanh lên. Hãy chọn giọng khác, hoặc tải file của bạn lên ở bên dưới.⚠️⚠️⚠️")
    audio_file="/content/model/vi_sample.wav"
else:
    audio_file = run_tts(vixtts_model,
            lang=language_code_map[language],
            tts_text=input_text,
            speaker_audio_file=reference_audio,
            normalize_text=normalize_text,
            verbose=verbose,
            output_chunks=output_chunks,)

from IPython.display import Audio
Audio(audio_file)

ModuleNotFoundError: No module named 'underthesea'

In [3]:
# @title 🎤 **Tải file âm thanh của bạn lên**
# @markdown Để đạt chất lượng tốt nhất, hãy tham khảo file '/content/model/vi_sample.wav'
# @
import os
import locale
from google.colab import files

denoise = True # @param {type:"boolean"}

# Upload the audio file
uploaded = files.upload()
for filename in uploaded.keys():
    # Convert the audio file to WAV format using ffmpeg
    uploaded_dir = os.path.dirname(filename)
    if denoise:
        !deepFilter "{filename}"
        !ffmpeg -i "{filename.replace('.wav', '_DeepFilterNet3.wav')}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
        os.remove(filename.replace('.wav', '_DeepFilterNet3.wav'))
    else:
        !ffmpeg -i "{filename}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
        os.remove(filename)
    break

from IPython.display import Audio, clear_output
clear_output()
print("> Đã tải file âm thanh lên")
Audio("/content/model/user_sample.wav")

Saving segment_12.wav to segment_12.wav
/bin/bash: line 1: deepFilter: command not found
segment_12_DeepFilterNet3.wav: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'segment_12_DeepFilterNet3.wav'

In [ ]:
# @title ⏬ **Lưu kết quả vào Drive**
# @markdown Chạy phần này để lưu kết quả vào Google Drive của bạn
# @markdown `/content/drive/MyDrive/vixtts-output`

from google.colab import drive
drive.mount('/content/drive')

# Save the output folder in "vixtts-output" in Google Drive, without overwriting existing files
!cp -n -r /content/output/* /content/drive/MyDrive/vixtts-output
print("> Đã lưu kết quả vào Google Drive")

In [ ]:
# @title ⚠️ **Dọn kết quả**
# @markdown Chạy phần này để xóa toàn bộ file trong `/content/output`
import shutil
shutil.rmtree('/content/output')
print("Đã xóa toàn bộ file trong /content/output")

In [1]:
# @title 📴 **Tắt Demo**
# @markdown Khi chạy xong thì bạn hãy tắt demo để tiết kiệm GPU nhé!
from google.colab import runtime
runtime.unassign()